In [25]:
import pickle
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
py.init_notebook_mode(connected=True) 

In [3]:
(drugmatrix_archive,pubchem_archive,ds1_archive,serotonin_archive) = \
  pickle.load(open('../data/unbalanced_data_results.pkl','rb'))

In [37]:
(drugmatrix_archive2,pubchem_archive2,ds1_archive2,serotonin_archive2) = \
  pickle.load(open('../data/unbalanced_data_results.sklearn.pkl','rb'))

In [40]:
data = []
# columns: [dataset, name, Kappa_init, ROC, Kappa_final, threshold]
for nm,v in drugmatrix_archive.items():
    init,final = v
    data.append(['drugmatrix',nm,init[1],init[-1],final[1],final[0]])
for nm,v in pubchem_archive.items():
    init,final = v
    data.append(['pubchem',nm,init[1],init[-1],final[1],final[0]])
for nm,v in ds1_archive.items():
    init,final = v
    data.append(['DS1',nm,init[1],init[-1],final[1],final[0]])
for nm,v in serotonin_archive.items():
    init,final = v
    data.append(['serotonin',nm,init[1],init[-1],final[1],final[0]])
df1 = pd.DataFrame(data,columns=['dataset', 'name', 'kappa_init', 'ROC', 'kappa_final', 'threshold'])

In [45]:
data = []
# columns: [dataset, name, Kappa_init, ROC]
for nm,v in drugmatrix_archive2.items():
    data.append(['drugmatrix',nm,v[0][1],v[0][-1]])
for nm,v in pubchem_archive2.items():
    data.append(['pubchem',nm,v[0][1],v[0][-1]])
for nm,v in ds1_archive2.items():
    data.append(['DS1',nm,v[0][1],v[0][-1]])
for nm,v in serotonin_archive2.items():
    data.append(['serotonin',nm,v[0][1],v[0][-1]])
df2 = pd.DataFrame(data,columns=['dataset', 'name', 'kappa_BRF', 'ROC_BRF'])

In [46]:
df = df1.merge(df2,on=['dataset','name'])
df.head()

,dataset,name,kappa_init,ROC,kappa_final,threshold,kappa_BRF,ROC_BRF
0,drugmatrix,CHEMBL1909215,0.000000,0.829830,0.416906,0.15,0.248659,0.989986
1,drugmatrix,CHEMBL1909211,0.365774,0.935224,0.751379,0.25,0.551367,0.990435
2,drugmatrix,CHEMBL1909210,0.000000,0.902642,0.281530,0.20,0.325429,0.987178
3,drugmatrix,CHEMBL1909209,0.307850,0.920535,0.610599,0.20,0.409567,0.992174
4,drugmatrix,CHEMBL1909204,0.000000,0.890900,0.384958,0.10,0.256293,0.985219


In [62]:
df['label'] = [x+"-"+y for x,y in zip(df.dataset,df.name)]

In [63]:
# Create a trace
pts = go.Scatter(
    x= df.kappa_init,
    y = df.kappa_final,
    mode = 'markers',
    text = df.label,
    name='data'
)
ln = go.Scatter( x=[0,1], y = [0,1], mode='lines',showlegend=False)

data = [pts,ln]

layout_comp = go.Layout(
    title='Kappas',
    hovermode='closest',
    xaxis=dict(
        title='Original Kappa'),
    yaxis=dict(
        title='Balanced Kappa'), 
)

# Plot and embed in ipython notebook!
fig_comp = go.Figure(data=data, layout=layout_comp)

py.iplot(fig_comp)

In [64]:
# Create a trace
pts = go.Scatter(
    x= df.kappa_final,
    y = df.kappa_BRF,
    mode = 'markers',
    text = df.label,
    name='data'
)
ln = go.Scatter( x=[0,1], y = [0,1], mode='lines',showlegend=False)

data = [pts,ln]

layout_comp = go.Layout(
    title='Kappas',
    hovermode='closest',
    xaxis=dict(
        title='Balanced Kappa'),
    yaxis=dict(
        title='BRF Kappa'), 
)

# Plot and embed in ipython notebook!
fig_comp = go.Figure(data=data, layout=layout_comp)

py.iplot(fig_comp)

In [74]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, ColorBar
from bokeh.palettes import plasma
from bokeh.plotting import figure
from bokeh.transform import transform, linear_cmap
output_notebook()

Loading BokehJS ...

In [76]:
source = ColumnDataSource(data=df)
hover = HoverTool(tooltips=[
    ("AUC", "(@ROC,@ROC_BRF)"),
    ("(x,y)", "(@kappa_init, @kappa_final)"),
    ('desc', '@label'),
])
mapper = linear_cmap(field_name='ROC',palette=plasma(256), low=min(df.ROC), high=max(df.ROC))

p = figure(plot_width=600, plot_height=400, tools=[hover], title="Kappas")
p.circle('kappa_init', 'kappa_final', size=7, source=source, color=mapper)
p.line((0,1),(0,1))
#         fill_color=transform('y', mapper))
p.xaxis.axis_label='Kappa(init)'
p.yaxis.axis_label='Kappa(balanced)'
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))

p.add_layout(color_bar, 'right')

#output_file('test.html')
show(p)

In [75]:

source = ColumnDataSource(data=df)
hover = HoverTool(tooltips=[
    ("AUC", "(@ROC,@ROC_BRF)"),
    ("(x,y)", "(@kappa_final, @kappa_BRF)"),
    ('desc', '@label'),
])
mapper = linear_cmap(field_name='ROC',palette=plasma(256), low=min(df.ROC), high=max(df.ROC))

p = figure(plot_width=600, plot_height=400, tools=[hover], title="Kappas")
p.circle('kappa_final', 'kappa_BRF', size=7, source=source, color=mapper)
p.line((0,1),(0,1))
#         fill_color=transform('y', mapper))
p.xaxis.axis_label='Kappa(balanced)'
p.yaxis.axis_label='Kappa(BRF)'
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))

p.add_layout(color_bar, 'right')

#output_file('test.html')
show(p)

In [77]:

source = ColumnDataSource(data=df)
hover = HoverTool(tooltips=[
    ("AUC", "(@ROC,@ROC_BRF)"),
    ("(x,y)", "(@kappa_init, @kappa_BRF)"),
    ('desc', '@label'),
])
mapper = linear_cmap(field_name='ROC',palette=plasma(256), low=min(df.ROC), high=max(df.ROC))

p = figure(plot_width=600, plot_height=400, tools=[hover], title="Kappas")
p.circle('kappa_init', 'kappa_BRF', size=7, source=source, color=mapper)
p.line((0,1),(0,1))
#         fill_color=transform('y', mapper))
p.xaxis.axis_label='Kappa(init)'
p.yaxis.axis_label='Kappa(BRF)'
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))

p.add_layout(color_bar, 'right')

#output_file('test.html')
show(p)